In [1]:
import datetime as dt
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

# Fixed parameters (number of samples)

In [2]:
#Number of samples determined by Chebyshev inequality
samples = 10000

# Variable parameters (qubits and layers)

In [3]:
#List of number of qubits
qubits = [4, 5, 6, 7, 8]

#List of number of layers
layers = [1, 2, 3, 4, 5]

# Function to calculate Meyer-Wallach-2 mean and standard deviation

In [4]:
def ent_cap(samples, circuit, nlayers):
    MW_mean = 0
    MW_mean_sqr = 0
    for i in range(samples):
        angles = np.array([ [[np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)], 
                            [np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)]] 
                            for j in range(nlayers)], requires_grad=True)
        
        G_measure = 0
        for j in range(1,wires):
            pur_sum = 0
            for k in range(wires-j):
                state = circuit(angles, k, j)
                pur_sum = pur_sum + np.trace(np.matmul(state, state))
    
            G_measure = G_measure + (wires-j)*( 4/3*(1-1/(wires-j)*pur_sum) )
        
        MW_measure = 2/(wires*(wires-1))*G_measure
        MW_measure_sqr = MW_measure**2
        
        MW_mean = MW_mean + MW_measure
        MW_mean_sqr = MW_mean_sqr + MW_measure_sqr
        
    MW_mean = MW_mean/samples
    MW_mean_sqr = MW_mean_sqr/samples
    MW_standev = np.sqrt(MW_mean_sqr - MW_mean**2)
    return(MW_mean, MW_standev)

# Code that executes the different conditions

In [5]:
now0 = dt.datetime.now() #monitorar tempo de início
now0 = now0.strftime("%Y-%m-%d %H:%M:%S")
print("Tempo inicial: ")
print(now0)
print()

for wires in qubits:
    dev = qml.device("default.qubit", wires=wires, shots=None)
    
    for nlayers in layers:
        
        #Identity circuit
        def identity(rots, k, j):
            for i in range(wires):
                qml.Identity(wires = [i])
            return qml.density_matrix([k,k+j])

        identity_circuit = qml.QNode(identity, dev)


        
        #No connections circuit
        def layer_noconnec(rots):
            for i in range(wires):
                qml.RX(rots[0][i], wires=[i])
                qml.RY(rots[1][i], wires=[i])
        
        
        def noconnec(rotations, k, j):
            qml.layer(layer_noconnec, nlayers, rotations)
            return qml.density_matrix([k, k+j])

        noconnec_circuit = qml.QNode(noconnec, dev)
        
        
        
        #Linear circuit
        def layer_linear(rots):
            for i in range(wires):
                qml.RX(rots[0][i], wires=[i])
                qml.RY(rots[1][i], wires=[i])
            qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")   

        def linear(rotations, k, j):
            qml.layer(layer_linear, nlayers, rotations)
            return qml.density_matrix([k, k+j])

        linear_circuit = qml.QNode(linear, dev)
        
        
        
        #Ring circuit
        def layer_ring(rots):
            for i in range(wires):
                qml.RX(rots[0][i], wires=[i])
                qml.RY(rots[1][i], wires=[i])
            qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")
            qml.CNOT(wires=[wires-1,0]) 

        def ring(rotations, k, j):
            qml.layer(layer_ring, nlayers, rotations)
            return qml.density_matrix([k, k+j])

        ring_circuit = qml.QNode(ring, dev)
        
        
        
        #Star circuit
        def layer_star(rots):
            for i in range(wires):
                qml.RX(rots[0][i], wires=[i])
                qml.RY(rots[1][i], wires=[i])
            for i in range(wires-1):
                qml.CNOT(wires = [0, i+1])

        def star(rotations, k, j):
            qml.layer(layer_star, nlayers, rotations)
            return qml.density_matrix([k, k+j])

        star_circuit = qml.QNode(star, dev)
        
        
        
        #All-to-all circuit
        def layer_alltoall(rots):
            for i in range(wires):
                qml.RX(rots[0][i], wires=[i])
                qml.RY(rots[1][i], wires=[i])
            qml.broadcast(qml.CNOT, wires=range(wires), pattern="all_to_all")
    
        def alltoall(rotations, k, j):
            qml.layer(layer_alltoall, nlayers, rotations)
            return qml.density_matrix([k, k+j])

        alltoall_circuit = qml.QNode(alltoall, dev)
        
        
        ent_identity, ent_identity_standev  = ent_cap(samples, identity_circuit, nlayers)
        ent_noconnec, ent_noconnec_standev = ent_cap(samples, noconnec_circuit, nlayers)
        ent_linear, ent_linear_standev = ent_cap(samples, linear_circuit, nlayers)
        ent_ring, ent_ring_standev = ent_cap(samples, ring_circuit, nlayers)
        ent_star, ent_star_standev = ent_cap(samples, star_circuit, nlayers)
        ent_alltoall, ent_alltoall_standev = ent_cap(samples, alltoall_circuit, nlayers)
        
        
        f = open("entanglement_MW_2_ansatz1_qubits{0}_layers{1}_samples{2}.txt".format(wires,nlayers, samples), "w")

        print('Circuit,MW_mean,MW_standdev', file = f)
        print('Identity,', ent_identity,',', ent_identity_standev, file = f)
        print('No connections,',ent_noconnec,',', ent_noconnec_standev, file = f)
        print('Linear,',ent_linear,',', ent_linear_standev, file = f)
        print('Ring,',ent_ring,',', ent_ring_standev, file = f)
        print('Star,',ent_star,',', ent_star_standev, file = f)
        print('All-to-all,',ent_alltoall,',', ent_alltoall_standev, file = f)
        
        print('Execution: qubits=',wires,'; layers=', nlayers)
        now1 = dt.datetime.now() #monitorar tempo de início
        now1 = now1.strftime("%Y-%m-%d %H:%M:%S")
        print("Tempo atual:",now1)
        print()

Tempo inicial: 
2023-06-27 15:24:47

Execution: qubits= 4 ; layers= 1
Tempo atual: 2023-06-27 15:59:40

Execution: qubits= 4 ; layers= 2
Tempo atual: 2023-06-27 17:04:58

Execution: qubits= 4 ; layers= 3
Tempo atual: 2023-06-27 18:39:28

Execution: qubits= 4 ; layers= 4
Tempo atual: 2023-06-27 20:43:58

Execution: qubits= 4 ; layers= 5
Tempo atual: 2023-06-27 23:20:00

Execution: qubits= 5 ; layers= 1
Tempo atual: 2023-06-28 00:30:52

Execution: qubits= 5 ; layers= 2
Tempo atual: 2023-06-28 02:45:53

Execution: qubits= 5 ; layers= 3
Tempo atual: 2023-06-28 06:04:50

Execution: qubits= 5 ; layers= 4
Tempo atual: 2023-06-28 10:28:35

Execution: qubits= 5 ; layers= 5
Tempo atual: 2023-06-28 15:55:47

Execution: qubits= 6 ; layers= 1
Tempo atual: 2023-06-28 18:03:07

Execution: qubits= 6 ; layers= 2
Tempo atual: 2023-06-28 22:05:37

Execution: qubits= 6 ; layers= 3
Tempo atual: 2023-06-29 04:05:41

Execution: qubits= 6 ; layers= 4
Tempo atual: 2023-06-29 12:04:33

Execution: qubits= 6 ; la

In [7]:
f.close()